In [16]:
import boto3





In [17]:
bucket="walstad-astronomer"
endpoint="Wh8CGNSjQeWBbC0Q1j8XEA"

In [18]:
def examine_harbr_endpoint(bucket, key, track):
    try:
        session = boto3.Session(profile_name='default')
        s3 = session.resource('s3')
        object = s3.Object(bucket, key)
    except BaseException as err:
        msg=f"Error while configuring S3 connection: {err=}, {type(err)=}"
        track.upload_failed(message=msg)
        raise

    track.record_start_time()
    try:
        singleload = s3.list_objects(Bucket=bucket, Prefix=key+"/"+"singleload/")
    except BaseException as err:
        msg=f"Error while acquiring a listing of the singleload folder: {err=}, {type(err)=}"
        track.upload_failed(message=msg)
        raise
    

In [19]:
name='name1'
value=42
params={"name":[],"value":[]}
params['name'].append(name)
params['value'].append(value)

params['name'].append('dict')
params['value'].append({"a":42,"b":42.0})

params

{'name': ['name1', 'dict'], 'value': [42, {'a': 42, 'b': 42.0}]}

In [20]:
import datetime

class tracking:
    def __init__(self, name):
        self.name=name
        self.parameters={'name':[],'value':[]}
        self.end_time = 0
        self.duration = 'Not set'
        self.success=False
        self.error_message="Not set"
        self.start_time = datetime.datetime.now()

        
    def record_parameter(self,name, value):
        self.parameters['name'].append(name)
        self.parameters['value'].append(value)
        
    def record_end_time(self):
        self.end_time = datetime.datetime.now()
        self.duration = self.end_time - self.start_time 
     
    def failed(self, message=""):
        self.end_time = datetime.datetime.now()
        self.duration = self.end_time - self.start_time 
        if(message!=""):
            self.error_message=message
        self.success=False
        
    def succeeded(self):
        self.end_time = datetime.datetime.now()
        self.duration = self.end_time - self.start_time 
        self.success=True
        
    def set_error_message(self, msg):
        self.error_message=msg
    
    def get_summary(self):
        summary = "Tracking: Name:       {}\n".format(self.name)
        summary+= "          Success:    {}\n".format(self.success)
        if(not self.success):
            summary+= "          Message:    {}\n".format(self.error_message)
        n=len(self.parameters['name'])
        if( n > 0 ):
            summary+= "          Parameters: Name \tValue\n"
            for i in range(0,n):
                summary+= "                      {}\t\t{}\n".format(self.parameters['name'][i],self.parameters['value'][i])
        
        summary+= "          Start:      {}\n".format(self.start_time) 
        summary+= "          End:        {}\n".format(self.end_time) 
        summary+= "          Duration:   {}\n".format(self.duration)
        
        return summary

        
track=tracking('test')
track2=tracking('failure test')
track.record_parameter('a',42.)
track2.record_parameter('a',42)
track2.record_parameter('l',['a','b','c'])
track2.failed("This is a test of failure.")
track.succeeded()
print(track.get_summary())
print("----------")
print(track2.get_summary())



Tracking: Name:       test
          Success:    True
          Parameters: Name 	Value
                      a		42.0
          Start:      2021-12-09 09:37:32.986798
          End:        2021-12-09 09:37:32.987069
          Duration:   0:00:00.000271

----------
Tracking: Name:       failure test
          Success:    False
          Message:    This is a test of failure.
          Parameters: Name 	Value
                      a		42
                      l		['a', 'b', 'c']
          Start:      2021-12-09 09:37:32.986920
          End:        2021-12-09 09:37:32.987026
          Duration:   0:00:00.000106



In [21]:
session = boto3.Session(profile_name='default')
s3 = session.client('s3')
folder="singleload"


In [22]:
import boto3

def check_harbr_endpoint_folder(s3,bucket,endpoint,folder,track):
    folder=endpoint+"/"+folder+"/"
    return list_key_prefix(s3, bucket, folder, track)

def list_key_prefix(s3,bucket,key,track):
    try:
        key_prefix_list = s3.list_objects_v2(Bucket=bucket, Prefix=key, MaxKeys=10)['Contents']
    except BaseException as err:
        msg=f"Unknown error: {err=}, {type(err)=}"
        track.failed(msg)
        key_prefix_list=None
    else:
        track.succeeded()
    
    return key_prefix_list


session = boto3.Session(profile_name='default')
s3 = session.client('s3')
bucket="walstad-astronomer"
endpoint="Wh8CGNSjQeWBbC0Q1j8XEA"
folder="singleload"




In [23]:
track=tracking("Check for singleload folder.")
contents=check_harbr_endpoint_folder(s3,bucket,endpoint,folder,track)
if(not track.success):
    print("Failed: \n{}")
print("{}".format(track.get_summary()))

Tracking: Name:       Check for singleload folder.
          Success:    True
          Start:      2021-12-09 09:37:37.088182
          End:        2021-12-09 09:37:37.414813
          Duration:   0:00:00.326631



In [24]:
print(contents)
print(len(contents))

[{'Key': 'Wh8CGNSjQeWBbC0Q1j8XEA/singleload/', 'LastModified': datetime.datetime(2021, 11, 12, 20, 12, 17, tzinfo=tzutc()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'Size': 0, 'StorageClass': 'STANDARD'}]
1


In [26]:
len(contents)
for  key in contents:
    print(key)

{'Key': 'Wh8CGNSjQeWBbC0Q1j8XEA/singleload/', 'LastModified': datetime.datetime(2021, 11, 12, 20, 12, 17, tzinfo=tzutc()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'Size': 0, 'StorageClass': 'STANDARD'}


In [25]:
track_setup=tracking()
try:
    session = boto3.Session(profile_name='default')
    s3 = session.client('s3')
except BaseException as err:
    msg=f"Error while configuring S3 connection: {err=}, {type(err)=}"
    track_setup.failed(message=msg)
    raise 
else:
    track_setup.succeeded()
#
# first, for the singleload folder and record if it has any content
#
folder="singleload"
check_harbr_endpoint_folder(s3,bucket,endpoint,folder,track)



TypeError: __init__() missing 1 required positional argument: 'name'

In [ ]:
singleload

In [14]:
object_list=singleload['Contents']
object_list

NameError: name 'singleload' is not defined

In [13]:
len(object_list)

NameError: name 'object_list' is not defined